Downloading Test Data

This dataset gives us ~42K text chunks to embed, each roughly a paragraph or two.

In [1]:
from datasets import load_dataset
# https://huggingface.co/datasets/jamescalam/ai-arxiv-chunked/viewer/default/train?p=0
# data = load_dataset(
#     "jamescalam/ai-arxiv-chunked",
#     split= "train")


data = load_dataset("json", data_files="data/train.jsonl", split="train")    
print(data.info)
print(type(data['chunk']),len(data['chunk'])) 
print(type(data['chunk'][0]),len(data['chunk'][0])) 


c:\code\on-prem_RAG_pgvector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetInfo(description='', citation='', homepage='', license='', features={'doi': Value(dtype='string', id=None), 'chunk-id': Value(dtype='string', id=None), 'chunk': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'source': Value(dtype='string', id=None), 'authors': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'categories': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'comment': Value(dtype='string', id=None), 'journal_ref': Value(dtype='string', id=None), 'primary_category': Value(dtype='string', id=None), 'published': Value(dtype='string', id=None), 'updated': Value(dtype='string', id=None), 'references': [{'id': Value(dtype='string', id=None)}]}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='json', dataset_name='json', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train',

## Creating Embeddings

create an embed function for each model -  return a list of vector embeddings from string list


### hugginface; load embedding model & embedding function to use with model

In [2]:
from embedding_models.hugginface import load_LLM


# model_id = "intfloat/e5-base-v2"
model_id = "C:/models/e5-base-v2"

model, tokenizer, device = load_LLM(model_id)

device: cpu


### Building a Vector Index

Use this to build a Numpy array of embedding vectors.

In [3]:
from embedding_models.hugginface import embed
from tqdm.auto import tqdm
import numpy as np
import time


def VectorIndex_in_memory(chunks,tokenizer= None,model= None,device = None):
    LEN = len(chunks)
    batch_size = 256
    for i in tqdm(range(0, LEN, batch_size)):
        # start = time.time()   
        chunk_batch = chunks[i:min(i+batch_size,LEN)].copy()
        # embed current batch
        embed_batch = embed(chunk_batch,tokenizer=tokenizer,model=model,device=device)
        if i > 0: # add to existing np array if exists (otherwise create)
            # embed_batch is the new batch of embeddings (same size as chunk_batch)
            # arr = np.concatenate([arr, embed_batch.copy()])
            arr += embed_batch.copy()
        if i == 0:
            arr = embed_batch.copy()
        # print(f'batch time: {time.time() - start}')
        if i>10:
            break
    print(i,len(arr) )   
    return arr
# arr = VectorIndex_in_memory(chunks = data["chunk"],tokenizer=tokenizer,model=model,device=device)




In [4]:
from add_data_to_db import VectorIndexUpdate
VectorIndexUpdate(texts= data["chunk"][:100],tokenizer=tokenizer,model=model,device=device)

100%|██████████| 50/50 [00:21<00:00,  2.30it/s]

finished writing Data to DB!


Now we need to create the query mechanism, this is simply a cosine similarity calculation between a query vector and our arr vectors.

In [5]:
# from numpy.linalg import norm

# # convert chunks list to array for easy indexing
# chunk_arr = np.array(chunks)

# def query_in_memory(text: str, top_k: int=3) -> list[str]:
#     # create query embedding
#     xq = embed([f"query: {text}"])[0]
#     # calculate cosine similarities
#     sim = np.dot(arr, xq.T) / (norm(arr, axis=1)*norm(xq.T))
#     # get indices of top_k records
#     idx = np.argpartition(sim, -top_k)[-top_k:]
#     docs = chunk_arr[idx]
#     for d in docs.tolist():
#         print(d)
#         print("----------")
# # query_in_memory("why should I use llama 2?")

In [6]:
from vector_search import query
texts = ["why should I use llama 2?"]
query(texts, model, tokenizer, device)



Input text: why should I use llama 2?
Similarity: 0.7536534335583621;	table ID: 324	Content: little to no technical skill. While this might make our paper seem harmful, we believe the beneﬁts of
publishing this attack far outweighs any potential harms.
The ﬁrst reason the beneﬁts outweigh the harms is that, to the best of our knowledge, multimodal
contrastive classiﬁers are not yet used in any security-critical situations. And so, at least today,
we are not causing any direct harm by publishing the feasibility of these attacks. Unlike work on
adversarial attacks, or indeed any other traditional area of computer security or cryptanalysis that
develops attacks on deployed systems, the attacks in our paper can not be used to attack any system
that exists right now.
Compounding on the above, by publicizing the limitations of these classiﬁers early, we can prevent
users in the future from assuming these classiﬁers are robust when they in fact are not. If we were
to wait to publish the feasi

## Usecases

https://medium.com/@vladris/embeddings-and-vector-databases-732f9927b377

https://medium.com/@vladris/n-shot-learning-f9bc0d670a41


Q&A solution

In [8]:
import json
import os

embeddings = {}

for f in os.listdir('./racing'):
    path = os.path.join('./racing', f)
    with open(path, 'r') as f:
        text = [f.read()]

    embeddings[path] = embed(text,tokenizer=tokenizer,model=model,device=device)[0]
   

with open('embeddings.json', 'w+') as f:
    json.dump(embeddings, f)

In [19]:
import json

embeddings = json.load(open('embeddings.json', 'r'))

def cosine_distance(a, b):
    return 1 - sum([a_i * b_i for a_i, b_i in zip(a, b)]) / (
        sum([a_i ** 2 for a_i in a]) ** 0.5 * sum([b_i ** 2 for b_i in b]) ** 0.5)
def nearest_embedding(embedding):
    nearest, nearest_distance = None, 1

    for path, embedding2 in embeddings.items():
        distance = cosine_distance(embedding, embedding2)
        if distance < nearest_distance:
            nearest, nearest_distance = path, distance

    return nearest#name of nearest file




i=0
while i<1:
    i+=1
    # prompt = input('user: ')
    prompt = 'What happened to Senn Kava during the Genosis Challenge?'
    if prompt == 'exit':
        break
    
    # find best context for prompt
    context = nearest_embedding(embed([prompt],tokenizer=tokenizer,model=model,device=device)[0])
    
    # open context - and ask new model question on given context
    data = open(context, 'r').read()
    print(data)
    # message = chat.completion(
    #     {'data': data, 'prompt': prompt}).choices[0].message
    # print(f'{message.role}: {message.content}')


During the Genosis Challenge Pod Racing race, there were several exhilarating and unforeseen events that shaped the final standings:

Lightning Bolt's Electrodynamic Boost: Tira Suro, piloting the Lightning Bolt, had equipped the pod with a cutting-edge electrodynamic propulsion system. As the race began, Suro ingeniously synchronized the pod's engine with the planet's unique electromagnetic field, harnessing its energy to achieve an unprecedented burst of speed. This electrifying boost propelled the Lightning Bolt into an early lead, setting the stage for Suro's victory.

Razor Blade's Risky Gambit: Kael Voss, the pilot of the Razor Blade, opted for a daring strategy to gain an advantage. Approaching a treacherous section filled with narrow rock formations, Voss executed a series of precise maneuvers, utilizing the Razor Blade's superior agility to navigate through the hazardous obstacles. Despite the risks involved, Voss's calculated moves allowed the Razor Blade to maintain a strong

In [22]:
import copy
import json
import openai
import os
import re
from langchain_community import LlamaCpp


# openai.api_key = None
# if openai.api_key is None:
#     raise Exception('OPENAI_API_KEY not set')

def insert_params(string, **kwargs):
    pattern = r"{{(.*?)}}"
    matches = re.findall(pattern, string)
    for match in matches:
        replacement = kwargs.get(match.strip())
        if replacement is not None:
            string = string.replace("{{" + match + "}}", replacement)
    return string

class ChatTemplate:
    def __init__(self, template):
        self.template = template

    def from_file(template_file):
        with open(template_file, 'r') as f:
            template = json.load(f)
        return ChatTemplate(template)

    def completion(self, parameters):
        instance = copy.deepcopy(self.template)
        for item in instance['messages']:
            item['content'] = insert_params(item['content'], **parameters)
        # return openai.ChatCompletion.create(
        #     model='gpt-3.5-turbo',
        #     **instance)
        return 
        model_path = 'C:/models/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q4_K_M.gguf'

        llm = LlamaCpp(model_path = model_path, temperature=0.5, max_tokens=500, top_p=0.9, top_k=50,)

        return  llm.invoke(instance)




chat = ChatTemplate( template=
    {'messages': [{'role': 'system', 'content': 'You are a Q&A AI.'},
                  {'role': 'system', 'content': 'Here are some facts that can help you answer the following question: {{data}}'},
                  {'role': 'user', 'content': '{{prompt}}'}]
     })
message = chat.completion(
    {'data': data, 'prompt': prompt})
# .choices[0].message
print(message)

ImportError: cannot import name 'LlamaCpp' from 'langchain_community' (c:\code\on-prem_RAG_pgvector\.venv\Lib\site-packages\langchain_community\__init__.py)

In [24]:
model_path = 'C:/models/Llama-2-7B-Chat-GGUF/llama-2-7b-chat.Q4_K_M.gguf'
from langchain_community import LlamaCpp
from llama_cpp import Llama
llm = LlamaCpp(model_path = model_path, temperature=0.5, max_tokens=500, top_p=0.9, top_k=50,)
# prompt = 


text = "What is the capital of France?"
response = llm.invoke(prompt.format(text = text))

ImportError: cannot import name 'LlamaCpp' from 'langchain_community' (c:\code\on-prem_RAG_pgvector\.venv\Lib\site-packages\langchain_community\__init__.py)